In [ ]:
import os
import json
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

#### Authentification INPI

In [ ]:
# --- Authentification INPI ---
load_dotenv()

USERNAME = os.getenv("INPI_USERNAME")
PASSWORD = os.getenv("INPI_PASSWORD")

if not USERNAME or not PASSWORD:
    raise RuntimeError("INPI_USERNAME ou INPI_PASSWORD manquant dans le .env")

LOGIN_URL = "https://registre-national-entreprises.inpi.fr/api/sso/login"

def login_inpi():
    payload = {"username": USERNAME, "password": PASSWORD}
    response = requests.post(LOGIN_URL, json=payload, timeout=10)
    response.raise_for_status()
    token = response.json()["token"]
    print("🔐 Token INPI obtenu")
    return {"Authorization": f"Bearer {token}"}

if "HEADERS" not in globals():
    HEADERS = login_inpi()

In [ ]:
RESULTS_FILE = "siren_attachments.json"
# --- Chargement des résultats existants ---
if os.path.exists(RESULTS_FILE):
    with open(RESULTS_FILE, "r") as f:
        results = json.load(f)
    print(f"📂 {len(results)} SIREN déjà présents")
else:
    results = {}
    print("📂 Aucun résultat existant")


In [ ]:
# --- Chargement de la liste des SIREN à traiter ---
with open("siren_list.json", "r") as f:
    siren_list = json.load(f)

print(f"✅ {len(siren_list)} SIREN chargés")
siren_list = [str(s) for s in siren_list]
results = {str(k): v for k, v in results.items()}

In [ ]:
# --- Calcul des SIREN restants à traiter ---
def has_bilan_exploitable(data):
    return any(b.get("bilanSaisi") for b in data.get("bilans", []))

remaining_siren = [
    s for s in siren_list
    if s not in results or not has_bilan_exploitable(results.get(s, {}))
]

print(f"ℹ️ {len(remaining_siren)} SIREN à récupérer")

---

#### Boucle de récupération

In [ ]:
# --- Paramètres ---
RESULTS_FILE = "siren_attachments.json"
BATCH_SIZE = 100
PAUSE_BETWEEN_REQUESTS = 5.0
PAUSE_BETWEEN_BATCHES = 60
MAX_RETRIES = 3
TIMEOUT = 10
BACKOFF_BASE = 10

# --- Boucle de récupération par batch ---
for i in range(0, len(remaining_siren), BATCH_SIZE):
    batch = remaining_siren[i:i + BATCH_SIZE]
    print(f"\n🔹 Batch {i//BATCH_SIZE + 1} / {(len(remaining_siren)-1)//BATCH_SIZE + 1}")

    for siren in tqdm(batch, desc="Récupération SIREN"):
        success = False
        attempts = 0

        while not success and attempts < MAX_RETRIES:
            try:
                url = f"https://registre-national-entreprises.inpi.fr/api/companies/{siren}/attachments"
                response = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
                response.raise_for_status()
                attachments = response.json()

                # 🔹 Ne garder que les clés utiles
                filtered = {
                    k: attachments.get(k, [])
                    for k in ["bilans", "comptesResultats"]
                    if k in attachments
                }

                # 🔒 Fusion sécurisée
                existing = results.get(siren, {})
                for key, value in filtered.items():
                    existing[key] = value
                results[siren] = existing

                success = True

            except requests.exceptions.HTTPError as e:
                code = e.response.status_code
                attempts += 1

                if code == 429:
                    wait = BACKOFF_BASE * (2 ** (attempts - 1))
                    print(f"⚠️ 429 pour {siren}, pause {wait}s")
                    time.sleep(wait)

                elif code == 401:
                    print(f"🔐 401 pour {siren} → refresh token")
                    HEADERS = login_inpi()
                    time.sleep(2)

                elif code == 404:
                    # Marque comme traité mais vide
                    results.setdefault(siren, {})
                    success = True

                else:
                    print(f"❌ {siren} erreur HTTP {code}")
                    time.sleep(5)

            except Exception as e:
                attempts += 1
                print(f"❌ {siren} erreur {e}")
                time.sleep(5)

        if not success:
            # trace pour éviter boucle infinie
            results[siren] = results.get(siren, {})

        time.sleep(PAUSE_BETWEEN_REQUESTS)

    # 💾 Sauvegarde intermédiaire (fin batch)
    with open(RESULTS_FILE, "w") as f:
        json.dump(results, f, indent=2)

    # 🔍 Contrôle de cohérence post-batch
    bilans_exploitables = {
        (s, b.get("numChrono"))
        for s, data in results.items()
        for b in data.get("bilans", [])
        if b.get("bilanSaisi")
    }
    print("📊 Bilans exploitables actuels :", len(bilans_exploitables))

    print(f"✅ Batch terminé — pause {PAUSE_BETWEEN_BATCHES}s")
    time.sleep(PAUSE_BETWEEN_BATCHES)

print("🎉 Récupération terminée")

In [ ]:
# Test de connection
siren_peugeot = "552100554"
url = f"https://registre-national-entreprises.inpi.fr/api/companies/{siren_peugeot}/attachments"

r = requests.get(url, headers=HEADERS, timeout=10)

print("HTTP status :", r.status_code)

---

#### Vérification des SIREN touchés par le 429 éventuel pour aller les chercher plus tard

In [ ]:
# --- 1. Charger les résultats existants ---
with open("siren_attachments.json", "r") as f:
    results = json.load(f)

# --- 2. Créer un DataFrame pour audit ---
audit = pd.DataFrame([
    {"siren": siren, "nb_bilans": len(data.get("bilans", []))}
    for siren, data in results.items()
])

# --- 3. Identifier les SIREN qui ont 0 bilan et qui sont donc probablement touchés par 429 ---
audit["probable_429"] = audit["nb_bilans"] == 0

# --- 4. Extraire la liste de SIREN à relancer ---
siren_a_relancer = audit.loc[audit["probable_429"], "siren"].tolist()

print(f"Nombre de SIREN touchés par 429 ou résultats vides : {len(siren_a_relancer)}")
print("Exemple des 10 premiers SIREN à relancer :")
print(siren_a_relancer[:10])